In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]='0,1'

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma


device = "cuda" if torch.cuda.is_available() else "cpu"
#Need only 1 GPU if loading 8-bit model
print(device)

print("Using %d GPUs" %torch.cuda.device_count())

import gradio as gr
import time


/home/beams/MCHERUKARA/miniconda3/envs/HugFace/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


cuda
Using 4 GPUs


In [2]:
model_name = "eachadea/vicuna-13b-1.1"
tokenizer_path = "./tokenizer/"

#Create a local tokenizer copy the first time
if os.path.isdir(tokenizer_path):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
else:
    tokenizer = AutoTokenizer.from_pretrained("model_name")
    os.mkdir(tokenizer_path)
    tokenizer.save_pretrained(tokenizer_path)

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")#, load_in_8bit=True)
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=1024,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
#Load embedding model and use that to embed text from source
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

with open("APS-Science-Highlight/2021-07-06_better-educated-neural-networks-for-nanoscale-3-d-coherent-x-ray.txt") as f:
    book = f.read()

#print(book)

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
texts = text_splitter.split_text(book)
docsearch = Chroma.from_texts(
    texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]
)

Using embedded DuckDB without persistence: data will be transient


In [6]:
def get_context(query):
    N_hits = 3 #How many hits of context to provide?
    docs = docsearch.similarity_search_with_score(query, k=N_hits)
    #Get context strings
    context=""
    for i in range(N_hits):
        context += docs[i][0].page_content +"\n"
    #print (context)
    return context

In [7]:
local_llm = HuggingFacePipeline(pipeline=pipe)


template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Context:
{context}

Current conversation:
{history}
Human: {input}
AI:"""

PROMPT = PromptTemplate(
    input_variables=["history", "input", "context"], template=template
)


memory = ConversationBufferWindowMemory(memory_key="history", 
                                        input_key = "input", 
                                        k=6)


conversation = LLMChain(
        prompt=PROMPT,
        llm=local_llm, 
        verbose=True, 
        memory=memory
)

#print(conversation.prompt.template)

In [8]:
with gr.Blocks(css="footer {visibility: hidden}", title="APS ChatBot") as demo:
    gr.Markdown("""
    # Welcome to the APS AI Assistant!
    Start typing below to see the output.
    """
    )
    chatbot = gr.Chatbot(show_label=False).style(height="500")
    msg = gr.Textbox(label="Send a message with Enter")
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):

        user_message = history[-1][0] #History is list of tuple list. E.g. : [['Hi', 'Test'], ['Hello again', '']]
        bot_message = conversation.predict(input=user_message, context=get_context(user_message))
        #Pass user message and get context and pass to model
        history[-1][1] = "" #Replaces None with empty string -- Gradio code

        for character in bot_message:
            history[-1][1] += character
            time.sleep(0.02)
            yield history
    

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: memory.clear(), None, chatbot, queue=False)

In [9]:
demo.queue()
demo.launch(server_name="0.0.0.0", server_port=2023)

Running on local URL:  http://0.0.0.0:2023

To create a public link, set `share=True` in `launch()`.


The 3D-CDI-NN model begins with an offline training stage in which polyhedral shapes are created by randomly clipping an fcc crystal lattice structure, applying and relaxing various stresses in the Large-scale Atomic/Molecular Massively Parallel Simulator (LAMMPS) molecular dynamics program, and voxelizing the data into a 32 x 32 x 32 voxel grid.  The grid is then entered into 3D-CDI-NN and its predictions matched with the known solutions.  The 3D-CDI-NN framework is quite accurate in its predictions, even when working with under-sampled diffraction patterns. 

To enhance the accuracy of 3D-CDI-NN even further, the investigators added a refinement that uses a reverse-mode automatic differentiation (AD) technique on the strain predictions and diffraction data.  This proved especially effective in recovering details of structural strain within the examined crystal samples, even when not using oversampled data, as is generally done when using AD techniques.
See: Henry Chan1,2*, Youssef S.

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/beams/MCHERUKARA/miniconda3/envs/HugFace/lib/python3.11/site-packages/starlette/responses.py", line 335, in __call__
    stat_result = await anyio.to_thread.run_sync(os.stat, self.path)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/beams/MCHERUKARA/miniconda3/envs/HugFace/lib/python3.11/site-packages/anyio/to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/beams/MCHERUKARA/miniconda3/envs/HugFace/lib/python3.11/site-packages/anyio/_backends/_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
           ^^^^^^^^^^^^
  File "/home/beams/MCHERUKARA/miniconda3/envs/HugFace/lib/python3.11/site-packages/anyio/_backends/_asyncio.py", line 754, in run
    re